<a href="https://colab.research.google.com/github/Angelvj/TFG/blob/main/code/jupyter/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import tensorflow as tf
import keras as k
import nibabel as nib
from sklearn.model_selection import train_test_split
from keras.datasets import mnist
from keras import models
from keras.models import Sequential
from keras.layers import Dense, Flatten, Conv3D, MaxPooling3D, Dropout, BatchNormalization
from keras.utils import to_categorical
import matplotlib.pyplot as plt

import time
import glob
from tqdm import tqdm

COLAB = True

if COLAB:
  from google.colab import drive
  drive.mount('/content/drive')
  DATA_PATH = '/content/drive/My Drive/Machine learning/data'

else: 
  DATA_PATH = '../../data'

def load_image(filename):    
    """
    
    Parameters
    ----------
    filename : str
        relative path to de image

    Returns
    -------
    img : numpy ndarray
        array containing the image
        
    """
    img = nib.load(filename)
    img = np.asarray(img.dataobj)
    img = np.expand_dims(img, axis=3)
    return img


def load_images_from_dir(dirname):
    """
    
    Parameters
    ----------
    dirname : str
        name of the directory containing images.

    Returns
    -------
    imgs : numpy ndarray
        array containing all of the images in the folder.

    """
    imgs = []

    for filename in tqdm(glob.glob(dirname + '/*.nii')):
        imgs.append(load_image(filename))
        
    imgs = np.stack(imgs)
    return imgs

def load_data(dirs_dict):
    """
    
    Parameters
    ----------
    dirs_dict : dictionary
        dictionary containing data folders name, and the label for the images
        on each forlder.

    Returns
    -------
    x : numpy ndarray
        array containing the images.
    y : numpy ndarray
        array containig the label of each image.

    """
    first = True
    for key, value in dirs_dict.items():
        if first:
            X = load_images_from_dir(value)
            y = np.full((X.shape[0]), key, dtype=np.uint8)
            first = False
        else:
            X_current = load_images_from_dir(value)
            X = np.concatenate((X, X_current))
            y = np.concatenate((y, np.full((X_current.shape[0]), key, dtype=np.uint8)))
            
    y = k.utils.to_categorical(y)
    
    return X, y

def impute_nan_values(imgs, inplace=True):
    return np.nan_to_num(imgs, copy= not inplace)

# Load PET images with labels
print('\n --- Loading PET data --- \n')
time.sleep(0.5)
X, y = load_data({0: DATA_PATH + "/ppNOR/PET", 
                  1: DATA_PATH + "/ppAD/PET",
                  2: DATA_PATH + "/ppMCI/PET"})

# Separate into training and test sets (stratified)
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size = 0.2, stratify = y, random_state = 1)


impute_nan_values(X_train)
impute_nan_values(X_test)

print('\n --- PET data loaded --- \n')

Mounted at /content/drive

 --- Loading PET data --- 



 27%|██▋       | 19/70 [00:10<00:39,  1.28it/s]

In [ ]:
# N-fold cross-validation


In [59]:
# 33% de cada tipo en test
print(np.sum(y_test, axis=0))



# basic 3D cnn model

# Model configuration
batch_size = 5
no_epochs = 10
learning_rate = 0.001
no_classes = 3
validation_split = 0.2
verbosity = 1
sample_shape = (79,95,68,1)

# Create the model --> accuracy del 60% en validación (parece que existe alguna información que aprender)
# model = Sequential()
# model.add(Conv3D(32, kernel_size=(3,3,3), activation='relu', kernel_initializer='he_uniform', input_shape=sample_shape))
# model.add(MaxPooling3D(pool_size=(2,2,2)))
# model.add(BatchNormalization()) # añadido
# model.add(Conv3D(32, kernel_size=(3,3,3), activation='relu', kernel_initializer='he_uniform'))
# model.add(MaxPooling3D(pool_size=(2,2,2)))
# model.add(Dropout(0.8)) # añadido
# model.add(Flatten())
# model.add(Dense(256, activation='relu', kernel_initializer='he_uniform'))
# model.add(Dense(no_classes, activation='softmax'))

# model = Sequential()
# model.add(Conv3D(32, kernel_size=(3,3,3), activation='relu', kernel_initializer='he_uniform', input_shape=sample_shape))
# model.add(MaxPooling3D(pool_size=(2,2,2)))
# model.add(BatchNormalization()) # añadido
# model.add(Conv3D(32, kernel_size=(3,3,3), activation='relu', kernel_initializer='he_uniform'))
# model.add(MaxPooling3D(pool_size=(2,2,2)))
# model.add(BatchNormalization()) # añadido
# model.add(Conv3D(32, kernel_size=(3,3,3), activation='relu', kernel_initializer='he_uniform'))
# model.add(MaxPooling3D(pool_size=(2,2,2)))
# model.add(BatchNormalization()) # añadido
# model.add(Conv3D(64, kernel_size=(3,3,3), activation='relu', kernel_initializer='he_uniform'))
# model.add(MaxPooling3D(pool_size=(2,2,2)))
# model.add(Flatten())
# model.add(Dropout(0.2)) # añadido
# model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
# model.add(Dropout(0.2))
# model.add(Dense(128, activation='relu', kernel_initializer='he_uniform'))
# model.add(Dense(no_classes, activation='softmax'))

# # Compile the model
# model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

# # Fit model to data
# history = model.fit(X_train, y_train, batch_size=batch_size, 
#                     epochs=no_epochs, verbose=verbosity, validation_split=validation_split)

# # Generate generalization metrics
# score = model.evaluate(X_test, y_test, verbose=0)
# print(f'Test loss: {score[0]} / Test accuracy: {score[1]}')

# # Plot history: Categorical crossentropy & Accuracy
# # plt.plot(history.history['loss'], label='Categorical crossentropy (training data)')
# # plt.plot(history.history['val_loss'], label='Categorical crossentropy (validation data)')
# plt.plot(history.history['accuracy'], label='Accuracy (training data)')
# plt.plot(history.history['val_accuracy'], label='Accuracy (validation data)')
# # plt.title('Model performance for 3D MNIST Keras Conv3D example')
# plt.ylabel('Loss value')
# plt.xlabel('No. epoch')
# plt.legend(loc="upper left")
# plt.show()


[14. 14. 22.]


In [ ]:

# sample_shape = X_train[0].shape

# # Basic 3D cnn model
# model = Sequential()
# model.add(Conv3D(32, kernel_size=(3, 3, 3), activation='relu', input_shape=(79,95,68,1)))
# model.add(MaxPooling3D(pool_size=(2, 2, 2)))
# model.add(BatchNormalization(center=True, scale=True))
# model.add(Conv3D(64, kernel_size=(3, 3, 3), activation='relu'))
# model.add(MaxPooling3D(pool_size=(2, 2, 2)))
# model.add(BatchNormalization(center=True, scale=True))
# model.add(Conv3D(128, kernel_size=(3, 3, 3), activation='relu'))
# model.add(BatchNormalization(center=True, scale=True))
# model.add(Conv3D(256, kernel_size=(3, 3, 3), activation='relu'))
# model.add(MaxPooling3D(pool_size=(2, 2, 2)))
# model.add(BatchNormalization(center=True, scale=True))
# model.add(Conv3D(256, kernel_size=(3, 3, 3), activation='relu'))

# model.add(BatchNormalization(center=True, scale=True))
# model.add(Conv3D(256, kernel_size=(3, 3, 3), activation='relu'))
# model.add(BatchNormalization(center=True, scale=True))
# model.add(Flatten())
# model.add(Dropout(0.2))
# model.add(Dense(512, activation='relu'))
# model.add(Dropout(0.2))
# model.add(Dense(512, activation='relu'))
# model.add(Dense(3, activation='softmax'))

# # Compile the model
# model.compile(loss='categorical_crossentropy',
#               optimizer=k.optimizers.Adam(lr=0.001),
#               metrics=['accuracy'])
# model.summary()
# # Fit data to model

# history = model.fit(X_train, y_train,
#             batch_size=8,
#             epochs=40,
#             verbose=1,
#             validation_split=0.2)

Model: "sequential_5"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv3d_24 (Conv3D)           (None, 77, 93, 66, 32)    896       
_________________________________________________________________
max_pooling3d_13 (MaxPooling (None, 38, 46, 33, 32)    0         
_________________________________________________________________
batch_normalization_23 (Batc (None, 38, 46, 33, 32)    128       
_________________________________________________________________
conv3d_25 (Conv3D)           (None, 36, 44, 31, 64)    55360     
_________________________________________________________________
max_pooling3d_14 (MaxPooling (None, 18, 22, 15, 64)    0         
_________________________________________________________________
batch_normalization_24 (Batc (None, 18, 22, 15, 64)    256       
_________________________________________________________________
conv3d_26 (Conv3D)           (None, 16, 20, 13, 128)  